<a href="https://colab.research.google.com/github/ni-metre/Colab_Triplet_Loss_Backup/blob/master/OldvNewClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')
% xmode verbose

Mounted at /content/drive
Exception reporting mode: Verbose


In [2]:
import pandas as pd
import numpy as np
import glob
import random
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns; sns.set()
import os
import datetime
import math
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict
#from Max_ML3 import prep_data, div_data, div_split_data, run_model, run_multi_model, run_experiment, concatenate_timeseries_and_static_features_list
import pickle
import time


In [25]:
sensor_of_interest = 15
base_name = 'V3_Fake7500_filled' + str(sensor_of_interest)

rootdir = '/content/drive/My Drive/V3_Fakes/'
#path to fake data
path = rootdir + base_name

pickle_path = '/content/drive/My Drive/Pickles'

#path to model checkpoints
checkpoint_path = '/content/drive/My Drive/Checkpoints'

#path to real data
real_path = '/content/drive/My Drive/General/test_outputs'

In [26]:
test_db_df=pd.read_csv(rootdir + 'Fake_Test_Log_for_' + base_name + '.csv')

#Convert to numeric to enable test sorting/picking
for column in test_db_df[['Acetone', 'CO2', 'H2O', 'Sensor', 'Test_that_day', 'Time_Btwn_s', 'Instr_Temp', 'Baseline', 'File', 'Total CO2']]:
    test_db_df[column] = pd.to_numeric(test_db_df[column])
test_db_df.fillna('', inplace = True)
test_db_df.set_index('File', inplace = True)
test_db_df.head(-1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10,11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Raw_data,Test_name,H2O,CO2,Total CO2,Acetone,Sample_rate,Sensor,Instr_Temp,Date,Time,Time_Btwn_s,Test_that_day,FLAG,Notes,Data_pts,Baseline,Amb_Temp,Other,Instr,Rel_Hum,EtOH_temp,Instr_T2,Instr_T3,New_Baseline,Baseline_Dip,Peak_neg,Peak_pos,Peak_neg_t,Peak_pos_t,Cross_t,Cross_t_ix,Peak_neg_t_ix,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,Area_total_X,Area_pos,...,Area_Pos_to_Cal_int,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,dx2_cross_t_ix.1,File_num
File,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10500,jdata10500,4% CO2 through plain water,10.0,40000.0,74.000,0.00000,250.0,15.0,25.3300,2020-12-15,9:44:19 AM,10000.00,1.00,,,14976,26.987831,0,,15,0,0,,,0.0,0,-119.561269,11.777794,2.408,14.212,9.220,1704,602,3553,0,0,-509,0,0,0,...,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,
10502,jdata10502,4% CO2 through plain water,10.0,40000.0,74.400,0.00000,250.0,15.0,25.3700,2020-12-15,9:53:06 AM,527.00,2.00,,,14976,27.094944,0,,15,0,0,,,0.0,0,-120.945881,11.897244,2.412,14.240,9.240,1707,603,3560,0,0,-516,0,0,0,...,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,
10504,jdata10504,5% CO2 through plain water,10.0,50000.0,74.900,0.00000,250.0,15.0,25.3500,2020-12-15,10:02:20 AM,554.00,3.00,,,14976,26.974712,0,,15,0,0,,,0.0,0,-151.532837,14.798725,2.420,14.524,9.320,1724,605,3631,0,0,-653,0,0,0,...,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,
10506,jdata10506,5% CO2 through plain water,10.0,50000.0,75.400,0.00000,250.0,15.0,25.4000,2020-12-15,10:11:04 AM,524.00,4.00,,,14976,27.063356,0,,15,0,0,,,0.0,0,-153.819606,15.296019,2.424,14.516,9.300,1720,606,3629,0,0,-663,0,0,0,...,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,
10508,jdata10508,6% CO2 through plain water,10.0,60000.0,76.000,0.00000,250.0,15.0,25.4000,2020-12-15,10:20:03 AM,539.00,5.00,,,14976,26.977687,0,,15,0,0,,,0.0,0,-186.214875,18.436375,2.420,14.552,9.380,1739,605,3638,0,0,-810,0,0,0,...,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45722,,Combo of 10670 and 10696 | Weight - 0.29,10.0,42900.0,118.285,0.90625,250.0,15.0,35.5771,,,543.74,5.77,,,12469,28.322912,,,15,,,,,0.0,,-209.960365,20.933932,2.224,13.480,8.624,2156,556,3370,,,-880,,,0,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
45723,,Combo of 10670 and 10696 | Weight - 0.77,10.0,47700.0,121.405,2.40625,250.0,15.0,35.5723,,,546.62,12.01,,,12469,28.385735,,,15,,,,,0.0,,-231.137310,23.668425,2.196,13.604,8.700,2175,549,3401,,,-982,,,0,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
45724,,Combo of 10670 and 10696 | Weight - 0.17,10.0,41700.0,117.505,0.53125,250.0,15.0,35.5783,,,543.02,4.21,,,12469,28.307206,,,15,,,,,0.0,,-204.675700,20.253678,2.232,13.344,8.600,2150,558,3336,,,-854,,,0,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [27]:
# Removes real test indices
test_db_df.index.get_loc(10752)

127

In [28]:
ixlist = np.array(test_db_df.index)[127:]
print(ixlist[:10])
print(len(ixlist))

[10752 10753 10754 10755 10756 10757 10758 10759 10760 10761]
34976


In [29]:
subset = random.sample(list(ixlist), 2500)
print(len(subset))
print(subset[:10])

2500
[42219, 20656, 14975, 37610, 34831, 27459, 16844, 36105, 19896, 28319]


In [32]:
cut_off_time = 30
sample_rate = 250

tco2 = []
ageLabel = []
csvlist = []
co2 = []
ac =[]
temp = []
bl = []
ttd = []
timebtwn = []
sensor = []
for val in subset:
  toco2val = float(test_db_df.loc[val, 'Total CO2'])
  if toco2val <= 100:
    ageLabelval = 0.0
  else:
    ageLabelval = 1.0
  tco2.append(test_db_df.loc[val, 'Total CO2'])
  ageLabel.append(ageLabelval)
  full_data = pd.read_csv(path+ '/test_output' + str(val) + '.csv',header = 0)
  sig_data = full_data[['Second','mV_rolling_mean']].truncate(after = cut_off_time*sample_rate-1)
  mV_data = np.array(sig_data['mV_rolling_mean'])
  csvlist.append(mV_data)
  ac.append(test_db_df.loc[val, 'Acetone'])
  co2.append(test_db_df.loc[val, 'CO2'])
  temp.append(test_db_df.loc[val, 'Instr_Temp'])
  bl.append(test_db_df.loc[val, 'Baseline'])
  ttd.append(test_db_df.loc[val, 'Time_Btwn_s'])
  timebtwn.append(test_db_df.loc[val, 'Test_that_day'])
  sensor.append(sensor_of_interest)
  if len(ageLabel) == 5:   
    print('5 done')
  if (len(ageLabel) % 100) == 0:    
    print(len(ageLabel))

5 done
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [31]:
foo = pd.read_csv('/content/drive/My Drive/V3_Fakes/V3_Fake7500_filled15/test_output42219.csv')
print(len(foo))

11401


In [34]:
testdict = defaultdict(list)
testdict['testnum'] = np.array(subset)
testdict['csvs'] = np.array(csvlist)
testdict['label'] = np.array(ageLabel)
testdict['CO2'] = np.array(co2)/2000
testdict['Ac'] = np.array(ac)
testdict['temp'] = np.array(temp)
testdict['baseline'] = np.array(bl)
testdict['ttd'] = np.array(ttd)
testdict['timebtwn'] = np.array(timebtwn)
testdict['sensor'] = np.array(sensor)


with open(pickle_path +'/oldvnew_2500_for_V3fake' + str(sensor_of_interest) + '.pickle', 'wb') as handle:
    pickle.dump(testdict, handle, protocol=pickle.HIGHEST_PROTOCOL)
